# Packages and Global variables

In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import numpy as np
import itertools
from collections import defaultdict
import time
from torchsummary import summary

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from PIL import Image, ImageFile
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler
from torchvision import datasets, transforms, utils
import snntorch as snn
from snntorch import surrogate
from snntorch import spikegen
import snntorch.spikeplot as splt
import math

torch.manual_seed(42)
np.random.seed(42)

#print(torch.cuda.is_available())

In [2]:
#data_path='/data/mnist'
data_path = '\\Users\\liamh\\OneDrive - University of Strathclyde\\University'
dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Training Parameters
batch_size=128

# Network Architecture
num_hidden = 350
num_outputs = 10
num_steps = 25

# Loss Function
loss_fn = nn.NLLLoss()  # Negative log-likelihood loss function
log_softmax_fn = nn.LogSoftmax(dim=-1) # Softmax activation for the output layer. -1 in 'dim' indicates last dimension (the labels.)


In [3]:
# Define Network
class Ann_Net(nn.Module):
    def __init__(self,res):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(res*res, num_hidden)     # input layer with as many neurons as pixels. 
        self.fc2 = nn.Linear(num_hidden, num_hidden)    # Second Dense/linear layer that receives the output spikes from previous layer
        self.fc3 = nn.Linear(num_hidden, num_outputs)
    
    def forward(self,x):
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        return(F.log_softmax(self.fc3(x),dim = 1)) # dim = 1 sums the rows so they equal 1. I.e. each input. 

def train_model(train_loader, valid_loader, model, epochs ,device = device, verbose = True):
    
    history = defaultdict(list)
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-4, betas=(0.9, 0.999)) # Just an Adam Optimiser
    
    # Training variables
    train_size = len(train_loader.dataset)
    train_num_batches = len(train_loader)
    
    # validation variables
    valid_size = len(valid_loader.dataset)
    num_batches = len(valid_loader)
    
    
    for t in range(epochs):
        correct = 0
        avg_valid_loss, valid_correct = 0, 0
        
        for batch, (X, y) in enumerate(train_loader):
            X = X.to(device)
            y = y.to(device)

            optimizer.zero_grad()
            # Compute prediction and loss
            pred = model(X)
            loss = loss_fn(pred, y)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            # Store loss history for future plotting

            # Backpropagation
            loss.backward()
            optimizer.step()


        history['avg_train_loss'].append(loss.item())
        avg_train_loss = loss / train_num_batches
        accuracy = correct / train_size * 100           
        history['train_accuracy'].append(accuracy)
        
        if verbose == True: 
            print(f"Epoch {t+1} of {epochs}")
            print('-' * 15)
            print(f"Training Results, Epoch {t+1}:\n Accuracy: {(accuracy):>0.1f}%, Avg loss: {avg_train_loss.item():>8f} \n")

              ###################### VALIDATION LOOP ##############################
        with torch.no_grad():
            for valid_X, valid_y in valid_loader:
                valid_X = valid_X.to(device)
                valid_y = valid_y.to(device)

                valid_pred = model(valid_X)
                valid_loss = loss_fn(valid_pred, valid_y).item()
                avg_valid_loss += loss_fn(valid_pred, valid_y).item()
                valid_correct += (valid_pred.argmax(1) == valid_y).type(torch.float).sum().item()
              
        avg_valid_loss /= num_batches
        valid_accuracy = valid_correct / valid_size * 100
              
        history['avg_valid_loss'].append(avg_valid_loss)
        history['valid_accuracy'].append(valid_accuracy)
        
        if verbose == True: 
            print(f"Epoch {t+1} of {epochs}")
            print('-' * 15)
            print(f"Validation Results, Epoch {t+1}: \n Accuracy: {(valid_accuracy):>0.1f}%, Avg loss: {avg_valid_loss:>8f} \n")


    print("Done!")
    print(f"Final Train Accuracy: {(accuracy):>0.1f}%, and Avg loss: {avg_train_loss.item():>8f} \n")
    print(f"Final Validation Accuracy: {(valid_accuracy):>0.1f}%, and Avg loss: {avg_valid_loss:>8f} \n")
    return history

def get_ann_results(resolution, epochs = 20, slope = 25, loss_upper = 1.05, acc_lower = 0, acc_higher = 100, verbose = True):
    train, valid, test = load_in_data(resolution)
    model = Ann_Net(resolution).to(device)

    output = train_model(train,valid,model,epochs, verbose = verbose)
    plot_training_history(output,resolution, ylimita = loss_upper, ylimitb_lower = acc_lower, ylimitb_upper = acc_higher)
    
    return output

# Functions

In [4]:
def load_in_data(res, ratio = 1):
    transform = transforms.Compose([
        transforms.Resize((res, res)), #Resize images to 28*28
        transforms.Grayscale(), # Make sure image is grayscale
        transforms.ToTensor()]) # change each image array to a tensor which automatically scales inputs to [0,1]

    mnist_train = datasets.MNIST(data_path, train=True, download=True, transform=transform) # Download training set and apply transformations. 
    mnist_test = datasets.MNIST(data_path, train=False, download=True, transform=transform) # same for test set

    train_len = int(len(mnist_train)/ratio)
    dummy_len = len(mnist_train) - train_len
    train_dataset, _ = random_split(mnist_train, (train_len, dummy_len), generator=torch.Generator().manual_seed(42))
    
    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True) # Load the data into the DataLoader so it's passed through the model, shuffled in batches. 
    test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=True, drop_last=True)
    
    return train_loader, test_loader

def output_formula(input_size, filter_size, padding, stride):
    formula = math.floor(((((input_size - filter_size + 2*padding)/stride) + 1)))
    
    return formula 

def all_output_sizes(res, conv_filter = 3, conv_padding = 1, conv_stride = 1, mp_filter = 3, mp_padding = 0, mp_stride = 2):
    
    conv1 = output_formula(res, conv_filter, conv_padding, conv_stride)   # Output size from applying conv1 to input 
    mp1 = output_formula(conv1, mp_filter, mp_padding, mp_stride)         # Output size from applying max pooling 1 to conv1 
    
    conv2 = output_formula(mp1, conv_filter, conv_padding, conv_stride)   # Output size from applying conv2 to max pooling 1
    conv3 = output_formula(conv2, conv_filter, conv_padding, conv_stride) # Output size from applying conv3 to conv 2
    mp2 = output_formula(conv3, mp_filter, mp_padding, mp_stride)         # Output size from applying max pooling 2 to conv3
    
    conv4 = output_formula(mp2, conv_filter, conv_padding, conv_stride)   # Output size from applying conv 4 to max pooling 2
    conv5 = output_formula(conv4, conv_filter, conv_padding, conv_stride) # Output size from applying conv5 to conv 4
    mp3 = output_formula(conv5, mp_filter, mp_padding, mp_stride)         # Output size from applying max pooling 3 to conv 5
    
    outputs_I_need = [mp1, conv2, mp2, conv4, mp3]
    
    return outputs_I_need

def plot_training_history(history, res, loss_upper = 1.05, acc_lower = -0.05, acc_higher = 105):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))
    
    
    ax1.plot(history['avg_train_loss'], label='train loss',marker = 'o')
    ax1.plot(history['avg_valid_loss'], label='validation loss',marker = 'o')

    ax1.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax1.set_ylim([-0.05, loss_upper])
    ax1.legend()
    ax1.set_ylabel('Loss',fontsize = 16)
    ax1.set_xlabel('Epoch',fontsize = 16)
    
    ax2.plot(history['train_accuracy'], label='train accuracy',marker = 'o')
    ax2.plot(history['valid_accuracy'], label='validation accuracy',marker = 'o')

    ax2.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax2.set_ylim([acc_lower, acc_higher])

    ax2.legend()

    ax2.set_ylabel('Accuracy',fontsize = 16)
    ax2.yaxis.set_major_formatter(PercentFormatter(100))
    ax2.set_xlabel('Epoch',fontsize = 16)
    fig.suptitle(f'Training history ({res}*{res})',fontsize = 20)
    plt.show()

def store_best_results(history):
    # Want to take the last entry from each output(best results) and store them all in a Dataframe
    placeholder = []
    placeholder.append(history['avg_train_loss'][-1])
    placeholder.append(history['train_accuracy'][-1])
    placeholder.append(history['avg_valid_loss'][-1])
    placeholder.append(history['valid_accuracy'][-1])
    
    return placeholder

In [5]:
def put_results_in_df(output):
    df = pd.DataFrame()
    df['avg_train_loss'] = output['avg_train_loss']
    df['train_accuracy'] = output['train_accuracy']
    df['avg_valid_loss'] = output['avg_valid_loss']
    df['valid_accuracy'] = output['valid_accuracy']
    
    return df

In [6]:
# Define Network
class Ann_Net(nn.Module):
    def __init__(self,res):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(res*res, num_hidden)     # input layer with as many neurons as pixels. 
        self.fc2 = nn.Linear(num_hidden, num_hidden)    # Second Dense/linear layer that receives the output spikes from previous layer
        self.fc3 = nn.Linear(num_hidden, num_hidden)
        self.fc4 = nn.Linear(num_hidden, num_outputs)
    
    def forward(self,x):
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        
        return(F.log_softmax(self.fc4(x),dim = 1)) # dim = 1 sums the rows so they equal 1. I.e. each input. 

In [7]:
def train_model(train_loader, valid_loader, model, epochs ,device = device, verbose = True):
    start_time = time.time()
    print('Starting Training')
    history = defaultdict(list)
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, betas=(0.9, 0.999)) # Just an Adam Optimiser
    
    # Training variables
    train_size = len(train_loader.dataset)
    train_num_batches = len(train_loader)
    
    # validation variables
    valid_size = len(valid_loader.dataset)
    num_batches = len(valid_loader)
    
    
    for t in range(epochs):
        correct = 0
        avg_valid_loss, valid_correct = 0, 0
        
        for batch, (X, y) in enumerate(train_loader):
            X = X.to(device)
            y = y.to(device)

            optimizer.zero_grad()
            # Compute prediction and loss
            pred = model(X)
            loss = loss_fn(pred, y)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            # Store loss history for future plotting

            # Backpropagation
            loss.backward()
            optimizer.step()


        history['avg_train_loss'].append(loss.item())
        avg_train_loss = loss / train_num_batches
        accuracy = correct / train_size * 100           
        history['train_accuracy'].append(accuracy)
        
        if verbose == True: 
            print(f"Epoch {t+1} of {epochs}")
            print('-' * 15)
            print(f"Training Results, Epoch {t+1}:\n Accuracy: {(accuracy):>0.1f}%, Avg loss: {avg_train_loss.item():>8f} \n")

              ###################### VALIDATION LOOP ##############################
        model.eval()
        with torch.no_grad():
            for valid_X, valid_y in valid_loader:
                valid_X = valid_X.to(device)
                valid_y = valid_y.to(device)

                valid_pred = model(valid_X)
                valid_loss = loss_fn(valid_pred, valid_y).item()
                avg_valid_loss += loss_fn(valid_pred, valid_y).item()
                valid_correct += (valid_pred.argmax(1) == valid_y).type(torch.float).sum().item()
              
        avg_valid_loss /= num_batches
        valid_accuracy = valid_correct / valid_size * 100
              
        history['avg_valid_loss'].append(avg_valid_loss)
        history['valid_accuracy'].append(valid_accuracy)
        
        if verbose == True: 
            print(f"Epoch {t+1} of {epochs}")
            print('-' * 15)
            print(f"Validation Results, Epoch {t+1}: \n Accuracy: {(valid_accuracy):>0.1f}%, Avg loss: {avg_valid_loss:>8f} \n")


    print("Done!")
    print(f"Final Train Accuracy: {(accuracy):>0.1f}%, and Avg loss: {avg_train_loss.item():>8f} \n")
    print(f"Final Validation Accuracy: {(valid_accuracy):>0.1f}%, and Avg loss: {avg_valid_loss:>8f} \n")
    current_time = time.time()
    total = current_time - start_time
    print(f'Training time: {round(total/60,2)} minutes')
    return history

def get_ann_results(resolution, ratio = 1,  epochs = 20, verbose = True):
    train, valid = load_in_data(resolution, ratio)
    model = Ann_Net(resolution).to(device)

    output = train_model(train,valid,model,epochs, verbose = verbose)
    
    return output

# Training models

## 56 * 56

In [8]:
batch_size = 128
output_56_r1 = get_ann_results(resolution = 56, ratio = 1, epochs = 75, verbose = False)

Starting Training
Done!
Final Train Accuracy: 99.8%, and Avg loss: 0.000000 

Final Validation Accuracy: 98.5%, and Avg loss: 0.152750 

Training time: 30.77 minutes


In [9]:
batch_size = 128
output_56_r4 = get_ann_results(resolution = 56, ratio = 4, epochs = 75, verbose = False)

Starting Training
Done!
Final Train Accuracy: 99.8%, and Avg loss: 0.000000 

Final Validation Accuracy: 97.3%, and Avg loss: 0.200885 

Training time: 12.52 minutes


In [10]:
batch_size = 32
output_56_r10 = get_ann_results(resolution = 56, ratio = 10, epochs = 75, verbose = False)

Starting Training
Done!
Final Train Accuracy: 99.7%, and Avg loss: 0.000000 

Final Validation Accuracy: 96.3%, and Avg loss: 0.328868 

Training time: 23.44 minutes


In [11]:
batch_size = 32
output_56_r100 = get_ann_results(resolution = 56, ratio = 100, epochs = 75,  verbose = False)

Starting Training
Done!
Final Train Accuracy: 96.0%, and Avg loss: 0.000004 

Final Validation Accuracy: 87.9%, and Avg loss: 0.757518 

Training time: 11.8 minutes


## 28 * 28

In [12]:
batch_size = 128
output_28_r1 = get_ann_results(resolution = 28, ratio = 1, epochs = 75,  verbose = False)

Starting Training
Done!
Final Train Accuracy: 99.8%, and Avg loss: 0.000000 

Final Validation Accuracy: 98.4%, and Avg loss: 0.141690 

Training time: 18.64 minutes


In [13]:
batch_size = 128
output_28_r4 = get_ann_results(resolution = 28, ratio = 4, epochs = 75,  verbose = False)

Starting Training
Done!
Final Train Accuracy: 99.8%, and Avg loss: 0.000000 

Final Validation Accuracy: 96.8%, and Avg loss: 0.247540 

Training time: 4.37 minutes


In [14]:
batch_size = 32
output_28_r10 = get_ann_results(resolution = 28, ratio = 10, epochs = 75, verbose = False)

Starting Training
Done!
Final Train Accuracy: 99.7%, and Avg loss: 0.000000 

Final Validation Accuracy: 95.3%, and Avg loss: 0.430565 

Training time: 6.18 minutes


In [15]:
batch_size = 32
output_28_r100 = get_ann_results(resolution = 28, ratio = 100, epochs = 75, verbose = False)

Starting Training
Done!
Final Train Accuracy: 96.0%, and Avg loss: 0.000012 

Final Validation Accuracy: 86.7%, and Avg loss: 0.790981 

Training time: 3.21 minutes


# 14 * 14

In [16]:
batch_size = 64
output_14_r1 = get_ann_results(resolution = 14, ratio = 1, epochs = 75,  verbose = False)

Starting Training
Done!
Final Train Accuracy: 99.9%, and Avg loss: 0.000000 

Final Validation Accuracy: 98.3%, and Avg loss: 0.116023 

Training time: 20.8 minutes


In [17]:
batch_size = 128
output_14_r4 = get_ann_results(resolution = 14, ratio = 4, epochs = 75, verbose = False)

Starting Training
Done!
Final Train Accuracy: 99.8%, and Avg loss: 0.000000 

Final Validation Accuracy: 97.3%, and Avg loss: 0.180026 

Training time: 5.19 minutes


In [18]:
batch_size = 32
output_14_r10 = get_ann_results(resolution = 14, ratio = 10, epochs = 75, verbose = False)

Starting Training
Done!
Final Train Accuracy: 99.7%, and Avg loss: 0.000000 

Final Validation Accuracy: 96.3%, and Avg loss: 0.259648 

Training time: 4.87 minutes


In [19]:
batch_size = 32
output_14_r100 = get_ann_results(resolution = 14, ratio = 100, epochs = 75, verbose = False)

Starting Training
Done!
Final Train Accuracy: 96.0%, and Avg loss: 0.000039 

Final Validation Accuracy: 88.6%, and Avg loss: 0.672056 

Training time: 2.89 minutes


## 7 * 7

In [20]:
batch_size = 128
output_7_r1 = get_ann_results(resolution = 7, ratio = 1, epochs = 75, verbose = False)

Starting Training
Done!
Final Train Accuracy: 99.7%, and Avg loss: 0.000002 

Final Validation Accuracy: 97.8%, and Avg loss: 0.115469 

Training time: 14.9 minutes


In [21]:
batch_size = 128
output_7_r4 = get_ann_results(resolution = 7, ratio = 4, epochs = 75, verbose = False)

Starting Training
Done!
Final Train Accuracy: 99.7%, and Avg loss: 0.000078 

Final Validation Accuracy: 96.2%, and Avg loss: 0.173870 

Training time: 6.09 minutes


In [22]:
batch_size = 32
output_7_r10 = get_ann_results(resolution = 7, ratio = 10, epochs = 75, verbose = False)

Starting Training
Done!
Final Train Accuracy: 99.5%, and Avg loss: 0.000083 

Final Validation Accuracy: 94.9%, and Avg loss: 0.297207 

Training time: 9.04 minutes


In [23]:
batch_size = 32
output_7_r100 = get_ann_results(resolution = 7, ratio = 100, epochs = 75, verbose = False)

Starting Training
Done!
Final Train Accuracy: 95.7%, and Avg loss: 0.000246 

Final Validation Accuracy: 87.7%, and Avg loss: 0.624996 

Training time: 4.71 minutes


# Saving Results

In [28]:
output_res = ['output_56', 'output_28', 'output_14','output_7']
output_ratio = ['_r1','_r4','_r10','_r100']
index = ['avg_train_loss', 'train_accuracy', 'avg_valid_loss', 'valid_accuracy']

In [25]:
all_columns = []
all_models = []
for name in output_res:
    for ratio in output_ratio: 
        model_name = name + ratio
        all_models.append(model_name)
        for indice in index: 
            column_name = name + ratio + '_' + indice
            all_columns.append(column_name)

In [26]:
df = pd.DataFrame()
for entry in all_models:
    for key in index:
        string = entry + '_' + key
        
        df[string] = locals()[entry][key]

In [27]:
df.to_csv('all_simple_ann_training_histories_final.csv')